In [29]:
import pandas as pd
import os
import json

In [2]:
movies_path = os.path.join(os.path.join(os.getcwd(),'Datasets'),'movies_dataset')

metadata_path = os.path.join(movies_path,'movies_metadata.csv')
ratings_path = os.path.join(movies_path,'ratings.csv')

In [6]:
metadata = pd.read_csv(metadata_path)
ratings = pd.read_csv(ratings_path, nrows=100)

In [43]:
genres = metadata.genres


genre_list = eval(genres[0])
print(genre_list[0])

for genre in genres:
    genre_list = eval(genre)
    for genre_dict in genre_list:
        name = genre_dict["name"]
        id_ = genre_dict["id"]
        
    break


{'name': 'Animation', 'id': 16}


In [7]:
ratings

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
95,4,3004,1.0,1042668521
96,4,3005,3.0,1042668544
97,4,3114,5.0,1042672580
98,4,3160,4.0,1042672335
